# `GridSearchCV`

Let's learn about a quick and easy way to conduct a cross-validation grid search without having to write a `for` loop.

## What we will accomplish

In this notebook we will:
- Work on a synthetic regression example,
- Refresh ourselves on hyperparameter tuning using cross-validation,
- Demonstrate what we would do using `KFold` and a `for` loop and
- Compare that to what we would do using `GridSearchCV`

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from seaborn import set_style

set_style("whitegrid")

## Hyperparameter tuning

Over the course of our supervised learning content we talked a lot about choosing the "best" hyperparameter values (think $\alpha$ in Ridge/Lasso regression, or $k$ in $k$NN) using cross-validation. What this entails is fitting the model with each potential set of hyperparameter values on each cross-validation split and then recording the performance on the holdout set.

Typically you <i>tune</i> hyperparameters, i.e. find the "best" values, by setting up grids for your potential hyperparameter values. For example, in a $k$NN setting you would set up a grid that starts at your minimum number of neighbors (like $k=1$) and incrementally increases to your maximum number of neighbors (like $k=50$).

### A synthetic regression example

Let's do just that with this synthetic data set.

In [ ]:
np.random.seed(403940)
X_train = np.random.randn(500,2)
X_train[:,0] = 1.3*X_train[:,0] - 2
X_train[:,1] = .8*X_train[:,1] + 1.2

y_train = X_train[:,0] + 2.3*X_train[:,1] - 2 + np.random.randn(500)*1.3

We will fit a $k$NN regression model, <a href="https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html">https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html</a>, on these data. In particular we will use a `for` loop and `KFold` to find the optimal values for `n_neighbors` and `weights`, where optimal indicates the pair with lowest average cross-validation mean squared error.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

For `n_neighbors` we will go from `1` to `50` and for `weights` we will examine `'uniform'` and `'distance'`, the two arguments allowed by `sklearn`.

In [ ]:
n_neighbors = range(1,51)
weights = ['uniform', 'distance']

## this array will track performance across all splits, n_neighbors, and weights
cv_mses = np.zeros((5, len(n_neighbors), len(weights)))

kfold = KFold(5, shuffle=True, random_state = 30293)

## keeps track of cv_split
i = 0
for train_index, test_index in kfold.split(X_train, y_train):
    X_tt = X_train[train_index, :]
    y_tt = y_train[train_index]
    X_ho = X_train[test_index, :]
    y_ho = y_train[test_index]
    
    ## keeps track of neighbor split
    j = 0
    for neighbors in n_neighbors:
        ## keeps track of weight split
        k = 0
        for weighting in weights:
            ## make the model object
            knn = KNeighborsRegressor(n_neighbors = neighbors,
                                         weights = weighting)
            
            ## fit the model
            knn.fit(X_tt, y_tt)
            
            ## get the prediction
            pred = knn.predict(X_ho)
            
            ## store the mse
            cv_mses[i,j,k] = mean_squared_error(y_ho, pred)
            
            k = k + 1
        j = j + 1
    i = i + 1

In [ ]:
np.mean(cv_mses, axis=0)

In [ ]:
## This will get you the index where the minimum occurs
np.unravel_index(np.mean(cv_mses, axis=0).argmin(), np.mean(cv_mses, axis=0).shape)

In [ ]:
## This is the min avg cv
np.mean(cv_mses, axis=0)[7,0]

In [ ]:
## this was the number of neighbors that got us there
n_neighbors[7]

In [ ]:
## this was the weighting
weights[0]

Now this example involed a two dimensional grid search, which was not too difficult to write up, but what if we wanted to test a random forest model which can involve tuning way more than two hyperparameters.

At a certain point, explicitly writing a `for` loop becomes tedious.

Luckily, `sklearn` provides a model selection object called `GridSearchCV`, <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html">https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html</a>, which will implement the grid search hyperparameter tuning for you.

Let's demonstrate how now.

In [ ]:
## import GridSearchCV
from sklearn.model_selection 

In [ ]:
## when defining a GridSearchCV object you first
## place in an empty model object,
## Then a dictionary containing the grids for each of the parameters you
## want to test into param_grid,
## Then a string with how you want to "score" the model in scoring,
## Then how many splits you want to use in cv, the default is 5
grid_cv = 


## Then you call to fit


When you call `.fit` the `GridSearchCV` object goes through the same `for` loop procedure we manually coded above and finds the `scoring` value for each of the grid points you have fed into the object.

When it is done we can access the results.

In [ ]:
## You can find the hyperparameter grid point that
## gave the best performance like so
## .best_params_
grid_cv

In [ ]:
## You can find the best score like so
## .best_score_
grid_cv

In [ ]:
## You can get all of the results with cv_results_
grid_cv

In [ ]:
## Calling best_estimator_ returns the model with the 
## best avg cv performance after it has been refit on the
## entire data set
grid_cv

In [ ]:
grid_cv

Before we end this notebook you may have a couple of questions.

First you may have noticed that our score was `'neg_mean_squared_error'` instead of `'mean_square_error'`. This is because the `GridSearchCV` does not offer mean squared error as an option.

To see what metrics are available as scoring options we can run the following code.

In [ ]:
from sklearn.metrics import SCORERS

In [ ]:
SCORERS.keys()

Another thing you may notice is that the model chosen by the `GridSearchCV` is not the same as the model we found with our handwritten `for` loop.

If you compare the performance, their avg cv scores are close, indicating that this is likely due to `GridSearchCV` using a different cross-validation split than ours. We can rectify that like so.

In [ ]:
grid_cv = GridSearchCV(KNeighborsRegressor(), 
                         param_grid = {'n_neighbors':range(1,50),
                                          'weights':['uniform', 'distance']}, 
                         scoring = 'neg_mean_squared_error',
                         cv = )

grid_cv.fit(X_train, y_train)

In [ ]:
grid_cv.best_params_

You now have a good base understanding of `GridSearchCV`. If you would like to learn more you can experiment on your own, or read the documentation, <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html">https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html</a>.

--------------------------

This notebook was written for the Erd&#337;s Institute C&#337;de Data Science Boot Camp by Matthew Osborne, Ph. D., 2022.

Any potential redistributors must seek and receive permission from Matthew Tyler Osborne, Ph.D. prior to redistribution. Redistribution of the material contained in this repository is conditional on acknowledgement of Matthew Tyler Osborne, Ph.D.'s original authorship and sponsorship of the Erdős Institute as subject to the license (see License.md)